# Analyzing Bobby Bradley
I'm not totally sure what the end result of this will be, but I wanted to mess with 2 new things, pybaseball, and jupyternotebooks.

I'll just be pulling interesting data and maybe adding some commentary about what I see.

In [11]:
import pandas as pd
import numpy as np
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
from pybaseball import statcast_batter_exitvelo_barrels
from pybaseball import cache

pd.get_option("display.max_columns")
cache.enable()

In [12]:
# we want key_mlbam
bobby = playerid_lookup("Bradley", "Bobby")
print(bobby)

  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0   bradley      bobby     656252  bradb002  bradlbo01          17278   

   mlb_played_first  mlb_played_last  
0            2019.0           2021.0  


In [13]:
# get pitch by pitch data for Bobby
full_data = statcast_batter('2021-04-01', '2021-09-01', player_id = 656252)
#print(full_data.columns.values.tolist())

In [14]:
# extract hit data from Bobby
hit_dataframe = pd.DataFrame(full_data, columns = ["pitch_type", "game_date", "events", "description", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "p_throws"])
# we only want balls that Bobby hit into play
contains_values_hit = hit_dataframe.loc[(hit_dataframe['description'] == "hit_into_play")]
# reset index count
contains_values_hit.reset_index(drop=True, inplace=True)
print(contains_values_hit)
hit_data = open("hit_data.txt", "w")
hit_data.write(contains_values_hit.to_string())
hit_data.close()

    pitch_type   game_date     events    description  \
0           CH  2021-09-01  field_out  hit_into_play   
1           FF  2021-09-01  field_out  hit_into_play   
2           FC  2021-08-31  field_out  hit_into_play   
3           FF  2021-08-31  field_out  hit_into_play   
4           CH  2021-08-31  field_out  hit_into_play   
..         ...         ...        ...            ...   
96          SI  2021-06-06     single  hit_into_play   
97          SI  2021-06-06     double  hit_into_play   
98          FF  2021-06-06   home_run  hit_into_play   
99          FF  2021-06-06  field_out  hit_into_play   
100         SI  2021-06-05     double  hit_into_play   

     estimated_ba_using_speedangle  estimated_woba_using_speedangle p_throws  
0                            0.120                            0.117        R  
1                            0.097                            0.167        L  
2                            0.007                            0.006        R  
3          

In [15]:
strikeout_dataframe = pd.DataFrame(full_data, columns = ["pitch_type", "game_date", "events", "description", "zone", "balls", "strikes", "release_spin_rate"])
contains_values_strikeout = strikeout_dataframe.loc[(strikeout_dataframe['events'] == "strikeout")]
contains_values_strikeout.reset_index(drop=True, inplace=True)
print(contains_values_strikeout)
strikeout_data = open("strikeout_data.txt", "w")
strikeout_data.write(contains_values_strikeout.to_string())
strikeout_data.close()

   pitch_type   game_date     events              description  zone  balls  \
0          CH  2021-09-01  strikeout            called_strike    11      1   
1          SL  2021-09-01  strikeout            called_strike     6      2   
2          FS  2021-08-29  strikeout          swinging_strike     5      0   
3          SI  2021-08-28  strikeout          swinging_strike     2      0   
4          SL  2021-08-28  strikeout          swinging_strike     4      0   
..        ...         ...        ...                      ...   ...    ...   
62         FF  2021-06-16  strikeout            called_strike     7      1   
63         FF  2021-06-15  strikeout          swinging_strike    11      1   
64         SL  2021-06-14  strikeout  swinging_strike_blocked     9      0   
65         SL  2021-06-13  strikeout          swinging_strike    14      0   
66         CH  2021-06-08  strikeout          swinging_strike    11      2   

    strikes  release_spin_rate  
0         2             2461.0

In [16]:
shift_dataframe = pd.DataFrame(full_data, columns = ["if_fielding_alignment", "of_fielding_alignment"])
#print(shift_data)

shift_data = open("shift_data.txt", "w")
shift_data.write(shift_dataframe.to_string())
shift_data.close()

In [17]:
contains_values_actual_hit = hit_dataframe.loc[(hit_dataframe['events'] == "single") | (hit_dataframe['events'] == "double") | (hit_dataframe['events'] == "triple") | (hit_dataframe['events'] == "home_run")]
contains_values_actual_hit.reset_index(drop=True, inplace=True)
print(contains_values_actual_hit)

contains_values_actual_hit.sort_values(by=['estimated_ba_using_speedangle'], ascending=False)

actual_hit_data = open("actual_hit_data.txt", "w")
actual_hit_data.write(contains_values_actual_hit.to_string())
actual_hit_data.close()

   pitch_type   game_date    events    description  \
0          FF  2021-08-31  home_run  hit_into_play   
1          FF  2021-08-28    single  hit_into_play   
2          FF  2021-08-28    single  hit_into_play   
3          SL  2021-08-06    double  hit_into_play   
4          CH  2021-08-06    single  hit_into_play   
5          SI  2021-08-04    single  hit_into_play   
6          FC  2021-08-03    single  hit_into_play   
7          CU  2021-08-03    double  hit_into_play   
8          CH  2021-07-24  home_run  hit_into_play   
9          SI  2021-07-18    single  hit_into_play   
10         KC  2021-07-10    double  hit_into_play   
11         SL  2021-07-09  home_run  hit_into_play   
12         SL  2021-07-08  home_run  hit_into_play   
13         SI  2021-07-08    single  hit_into_play   
14         FF  2021-07-05    single  hit_into_play   
15         FF  2021-07-03    double  hit_into_play   
16         KC  2021-07-02    single  hit_into_play   
17         SI  2021-06-30  h

Lowest Probability Hit:

30         SL  2021-06-11    single  hit_into_play                          0.080

https://streamable.com/bnwrki

Highest Probability Hits:

8          CH  2021-07-24  home_run  hit_into_play                          0.943

https://streamable.com/817wjg

18         CH  2021-06-30  home_run  hit_into_play                          0.943

https://streamable.com/t2cdsi

# Analyzing the strikeout data we have extracted
Let's count his most common strikeout pitches

Here's a little glossary for pitch abbreviations
FA = fastball
FF = four-seam fastball
FT = two-seam fastball
FC = fastball (cutter)
FS / SI / SF = fastball (sinker, split-fingered)
SL = slider
CH = changeup
CB / CU = curveball
KC = knuckle-curve
KN = knuckleball
EP = eephus
UN / XX = unidentified
PO / FO = pitch out

In [18]:
contains_values_strikeout["pitch_type"].value_counts()


FF    15
CH    13
SL    13
SI    10
FC     6
CU     5
FS     4
KC     1
Name: pitch_type, dtype: int64

Fastballs: 35 pitches
Offspeed: 32 pitches

It doesn't appear as if pitch type alone is something that causes Bobby to strike out more or less this year

In [19]:
contains_values_strikeout["zone"].value_counts()


14    12
11     9
7      8
13     7
9      6
1      5
2      4
4      4
12     4
3      3
8      3
6      1
5      1
Name: zone, dtype: int64

Here is our possible culprit, pitch location!

Nearly a whopping 60% (59.7%) of his strikeouts come from the zones outside of the called strike zone.

Now, obviously good pitchers will get you to swing outside of the called zone, but better plate discipline is certainly something that can be taught and developed.

In [20]:
vs_pitcher_handedness_dataframe = pd.DataFrame(full_data, columns = ["pitch_type", "game_date", "events", "description", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "p_throws", "woba_value"])
#print(vs_pitcher_handedness_dataframe)
vs_pitcher_handedness = vs_pitcher_handedness_dataframe.loc[(vs_pitcher_handedness_dataframe['p_throws'] == "R")]
vs_pitcher_handedness.reset_index(drop=True, inplace=True)

print(vs_pitcher_handedness)

    pitch_type   game_date     events      description  \
0           CH  2021-09-01  field_out    hit_into_play   
1           CH  2021-09-01  strikeout    called_strike   
2           FF  2021-09-01       None  swinging_strike   
3           CH  2021-09-01       None  swinging_strike   
4           FF  2021-09-01       None             ball   
..         ...         ...        ...              ...   
499         CH  2021-06-05       None             foul   
500         SI  2021-06-05       None             foul   
501         SI  2021-06-05       None             foul   
502         SI  2021-06-05       None  swinging_strike   
503         SI  2021-06-05       None             ball   

     estimated_ba_using_speedangle  estimated_woba_using_speedangle p_throws  \
0                             0.12                            0.117        R   
1                              NaN                              NaN        R   
2                              NaN                             